# 문제점


예측문제 / 분류문제

(여기서 x 값은 독립변수 y 값은 종속변수.)

처음엔 결측치, 이상치를 판단후에 제거할지 mean처리할지 판단
그 뒤에 X값끼리 상관관계를 분석후에 다중공선성을 판단해야함.
서로 상관관계가 높다고 나오면 다중 공선성이 있기때문에 제거해야함.

그리고 y 값하고 x 값이 상관관계가 있어야 한다고 판단되어야한다 (0.5 이상.)
종속변수를 예측해야함으로 즉 x 값과 y 값이 상관관계가 있어야 합니다.
data.corr()

기본적으로 Traing 세트하고 Test 세트가 나눠져있는것이 바람직합니다.
기본적으로 모델링뒤에 Regression 이 붙으면 다중회귀, Classifier 가 붙으면 분류기입니다.
전부다 파라미터가 다르니 Shift + Tab 으로 파라미터를 확인 해야합니다.

X,y 값이 정해져있으면 MSE 와 RMSE 로 크기에 의존하여 에러를 확인하는방법을 사용
X 값을 scale 해주고서
예측모델에 fit해준뒤 
predict으로 테스트값을 테스팅해준다음
mse 값과 rmse 를 출력후 한번 검증
y,y_pred 값 넣어주자

MSE가 0에 가까울수록 정확도가 높은것이다.

회귀식 y = a*x + b 가 나오는건
LinearRegressor 만 나옵니다.



기본 순서도

In [1]:
# 기본 예측문제에 사용되는 패키지
import pandas as pd
import numpy as np

# Traing / Test 세트를 나눠줄 패키지
from sklearn.model_selection import train_test_split 
# 전처리하는 세트
from sklearn.preprocessing import scale, robust_scale, minmax_scale , maxabs_scale
from sklearn.preprocessing import MinMaxScaler # MinMax 점수
from sklearn.preprocessing import StandardScaler # Z 점수

# 다중회귀분석을 하기 위한 패키지
from sklearn.linear_model import LinearRegression # 이놈만 coef_ / intercept_ 가 나옴. y = a*x + b / coef는 x / intercept는 b
from sklearn.linear_model import Lasso,Ridge,Hinge # 규제모델
from sklearn.svm import SVR # Support Vector Regressor 차원의 수를 늘려서 모델을 만듬.
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor # 여러모델을 상대로 투표방식으로 재검증
from sklearn.tree import DecisionTreeRegressor # 결정트리.
from sklearn.neighbors import KNeighborsClassifier


# 검증 모델~
from sklearn.metrics import mean_squared_error # MSE / RMSE
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict # 폴더수로 트레인/ 테스트
#cross는 여러개 크로스해서 트레인셋이 테스트가되고 테스트가 트레인이 됨.


# 모델 selection / 여러 모델들의 파라미터를 수정하여 최적의 파라미터를 정해줌!
from sklearn.model_selection import GridSearchCV


In [ ]:
# 제일 기본적인 회귀식 만들기.

scaler = scale(x)

lig_clf = LinearRegression().fit(scaler,y)

y_pred = lig_clf.predict(scaler)

mse = mean_squared_error(y,y_pred)
rmse = mean_squared_error(y,y_pred)**0.5

# 문제확인
print("MSE : ",mse)
print("RMSE : ",rmse)

# y = ax + ax1 + ax2 + b
print(lig_clf.coef_) # 가중치/기울기 = 계수, 각 곱해주는값 a값
print(lig_clf.intercept_) # b값


In [ ]:
import numpy as np
from sklearn.decomposition import PCA


# 16개의 변수중 영향력있는 변수 3개추출
pca = PCA(n_components=16)
model = pca.fit_transform(minmax_X)
print(pca.explained_variance_ratio_) # 설명력

plt.plot(np.cumsum(pca.explained_variance_ratio_)) # elbow를 시각적으로 확인
plt.show()
plt.plot(pca.explained_variance_ratio_, 'bo-')
plt.show()